<h1> General introduction </h1>
Today, you will assemble and annotate a metagenome of a plant: Azolla filiculoides. Azolla is known for its endophytes: microorganisms that live inside the plant. These microorganisms live specifically inside a leaf cavity. We are going to investigate which species live inside these leaf cavities and infer about their function. Your goal is to assemble the genomes of these endophytes from a metagenomic dataset, then sort them in taxonomical operational units and present information on their metabolic capacity.



<h2> Workflow </h2>
A metagenome assembly is different than a regular genome assembly in that it has DNA of multiple species in one sequencing file. Since we are interested in a metagenome associated to a host, we must also filter out sequencing data concerning Host DNA. This step, we have already performed for you. Since you are now familiar with handling FastQ files, quality controll, trimming and assembling a genome, we will not cover this part of the metagenome assembly. 
Here follows a short overview of the workflow you will perform today.
![workflow](metagenomics/workflowsketch.png)



We preinstalled the following programs on the virtual machine: <br>
bwa: http://bio-bwa.sourceforge.net/ <br>
samtools: http://samtools.sourceforge.net/ <br>
binningmetabat: https://bitbucket.org/berkeleylab/metabat/overview <br>
checkm: https://github.com/Ecogenomics/CheckM/wiki/Installation#how-to-install-checkm <br>
prokka: http://www.vicbioinformatics.com/software.prokka.shtml <br>


<h3> Assembly </h3>
For the assembly two sample types, plant and leaf, were sequenced both in three biological replicates. True endophytes are expected to be more abundant in leaf samples than in plant samples. The sequencing data was rid of any plant DNA, this was done by mapping (aligning) the sequencing data (reads) to reference plant genomes. Only the sequencing data that did not map anything was kept for further analysis, and pooled. These pools were then assembled using SPAdes. Your first assignment is to collect this assembly and assess its length distribution. But first read on:

The process of assembly takes the millions of short reads in a fastq file, and processes these to thousands of longer nucleotide sequences. Typically the output is a fasta file with contiguous sequences called contigs (contigs.fasta) or a fasta file with multiple contigs scaffolded based on their relative position and orientation to each other called scaffolds, in a file scaffolds.fasta. Ideally, these scaffolds represent the genome of a single species in the metagenome. This is however, extremely unlikely. At least in my personal experience. In reality the assembly process wil produce thousands of longer scaffolds where a single species' genome is still represented by multiple sequences in the assembly fasta file.

<h3> Binning </h3>
To tackle this issue, we will categorise each individual scaffold in a 'bin' which then ideally comprises one species in the metagenome. This procedure is called binning. The binning procudure starts with 'backmapping'. This means mapping the original reads from each biological replicate separately to the assembly. For this, we use BWA (Burrows-wheeler alignment) which then produces a bam file containing all reads and their location on the scaffolds. These bam files then must be sorted for further processing. Then metabat uses these sorted bamfile to subdivide the scaffolds in bins. This will produce between 5 and 20 bins. Hence we have moved from millions of very short sequences with very little biological information (to lets say about) 10 long sequences with a lot of biological information. 

<h3> Annotation </h3>
One bin must then ideally represent one species. The check wether this is the case, we can use checkM. CheckM assesses the completeness and contamination of bins based on marker genes (collocated sets of genes that are ubiquitous and single-copy within a phylogenetic lineage). 

For the sake of simplicity and time, we will isolate one bin per group and annotate this bin using prokka. Annotation is the process of adding functional information to parts of DNA sequence, i.e. making a table of genes and locations of these genes. These are often stored in a GFF file, the genomic feature format. Once you have this gff file, and a fasta for your specific bin/microbial species, we will visualise it using KEGG. Again, for more details please google kegg. Here we will use it to load our gff file and then explore the metabolic pathways present. 

<h2> Practical information </h2>
This whole computer practicum will take a whole day, staff will be present all day to help you in this process. However, we are with very few so please assisst each other where possible. 

You will work in this python notebook, combining bash and python commands. Bash commands can be executed by appending an exclamation mark '!' to the command like so:

> !whoami

Or by typing "%%bash" at the start of your code block, like so:

> %%bash


You are working on the virtual machine, which you have used before, did you know you can check the computer power of this machine using bash commands:

number of cpu threads

> !nproc

available GB RAM memory

> !top


The first bash commands we will use are 'mkdir' 'wget' and 'ls'. Go ahead and see what they do. If you wonder, most programms print a short summary of their function when presented the -h flag like so:

> !ls -h

or 

> !ls --help

or 

> !mkdir -h


If you made a directory you want to remove again you can use:

> !rm -rf directory

but be carefull with the rm command because you will not be able to recover the removed file.



Not all steps are feasible on these humble web based machines. And some may fail for other reasons. Therefore, results of all steps are uploaded to a separate web server. Instructions on accessing this webserver will follow .


Good luck! and remember Google is your friend!!!



## Raw data

<b> sequencing reads </b><br>
The sequencing reads are already available in the folder 'metagenomics/reads', to keep the practical feasible we have made a subset of the original data. <br>

In this reads folder you will find 12 files. <br>
L and P stand for the leaf and plant samples. <br>
L1 L2 L3 are the biological replicates. R1 and R2 represent the forward and reverse illumina reads. So 2 samples * 3 replicates * 2 directions = 12 files <br>
Files are stored in a fastq format (.fastq) and compressed in gzip archives (.gz). The gzip compression format is widely used in the genomics field, fastq files often compress down to only a quarter of the filesize with gzip! You do not need to extract these. <br>

A good practice is to always peek into a file just to check the format. 

This command allows you to see the files in the reads directory.   'ls' is short for 'list'

In [7]:
!ls metagenomics/reads/

L1.R1.fastq.gz	L2.R2.fastq	L3.R2.fastq  P2.R1.fastq  P3.R2.fastq
L1.R2.fastq.gz	L2.R2.fastq.gz	P1.R1.fastq  P2.R2.fastq
L2.R1.fastq.gz	L3.R1.fastq	P1.R2.fastq  P3.R1.fastq


To view in a gzipped file you can use the zcat command. 

The '|' sign is called a 'pipe' You can use it to feed the output of one programme to the next. Never forget to pipe the output to head when viewing/reading a file, like this "| head". The files we work with are big and will crash your jupyter-notebook instance if you don't use head.

In [8]:
!zcat metagenomics/reads/L1.R1.fastq.gz | head

@NS500813:28:H3M3VAFXX:1:11101:21566:1019 1:N:0:CAGATC
AAAAAATTAAAATAATTGGCTGAAAATGAAAAAAGAAAGCTTGATCATATGTTAAGCCTTTTGTAACGGAACAAAGTGATTAAATTATCTCTACAGTGTTTTTTCGATTTATTGATGGATTAAGAACAACCGTCACACACCAATAGAATAT
+
AAAAAEEEE6EEEEEEEEEEEEEEEEEEEEEAEEEEEEEEE</EE<EEEEEEEE/AA6E<E/E/</EEEAE/EEEEEEE<EEE//EEEEE<EE/EEAE<E<E//AE/E<EE/<AEEEAE66<E<<E</<AA<E/AEE/E//<<EEEEE/AA
@NS500813:28:H3M3VAFXX:1:11101:14840:1067 1:N:0:CAGATC
GTGGGATTTCCAGTGATTTGAATTCTTACAAAAGTCGCTTGCTTTTGTTTTTATTGCCACCTATTTCGTTGAATAATTATTAGAAAGGGAACAAGGCTTTTTGACGATAAAAAGTTATTTTTAATTGGAGGTCGCTAGAATAGCTTTGTT
+
A/AAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEAEEAEEEEEEEEEEEEEAEEE/EE/EEEEEEAEEEEEEEEEAEEEEEEEEEA/EEEAEAA<EEEEEEEEEAAEA/A/AA/6A<E<AE/EA6<A
@NS500813:28:H3M3VAFXX:1:11101:20510:1074 1:N:0:CAGATC
CCCGGTGCTGAATGAGAATGATTCGTGTTTAGATTAGCAGGCGCTGATGGTAGTAGTAGCGGCAATGAGAGTGATTCGTATTTAGATCCAGGCAGGGGCGGAAGTTTCATAATGTGGAATGCGCCCTGGTGCGCTTTACAATGTAAAGC

gzip: stdout: Broken pipe


Using grep we can select for lines in a file that contain only the word/key specified in the grep command. As you can see in the above command, the headers of the reads always start with the "@", therefor if we only want to see the headers we can simply grep for the "@" using the command below:

In [9]:
!zcat metagenomics/reads/L1.R1.fastq.gz | grep '@' | head

@NS500813:28:H3M3VAFXX:1:11101:21566:1019 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:14840:1067 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:20510:1074 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:5773:1086 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:2702:1014 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:6665:1015 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:6357:1086 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:18737:1028 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:1677:1028 1:N:0:CAGATC
@NS500813:28:H3M3VAFXX:1:11101:26277:1020 1:N:0:CAGATC
grep: write error: Broken pipe

gzip: stdout: Broken pipe


Now if we want to count the number of reads in the fastq file we can count the number of headers, since every entry has one header. To count the number of headers we will first have to read in the gzip file, then grep on the headers, and then we can use the wordcount command to count the number of lines.

In [10]:
!zcat metagenomics/reads/L1.R1.fastq.gz | grep '@' | wc -l

1000000


### wrap up
So now you have executed some bash commands and investigated the output. 

## Assembled data

Calculating a de-novo metagenome assembly is beyond the scope of this practical, so we have done this for you. You will however analyse this assembly yourself! :)

View the inside of the assembly directory:

In [13]:
!ls metagenomics/assembly/

scaffolds.fasta.gz


Now your first assignment is to dowload the assemblies from the data repository, adapting the commands above. Below you will find a code block with the template command that you can finish to download the the assemblies. Use multiple code blocks if convenient. You can make new code blocks by clicking on the left side of a code block and then type "a" or "b" to create a code block above or below the original code block. You can delete a code block by clicking on the left side of a code block and typing "d" twice.

Edit this line to have a look into the assembly file

In [ ]:
!zcat metagenomics/assembly/....assemblyfile | head

Grep the headers of the assembly file, what word/key would work for fastas when viewing the assembly entries above

In [ ]:
!zcat metagenomics/assembly/....assemblyfile | grep "" | head

Count the scaffolds in the assembly like we did earlier with the fastq.gz file. 

Remember not to over-load the webinterface!

In [ ]:
!zcat metagenomics/assembly/....assemblyfile | grep "" ......

Rename the assembly file:   (mv is short for move)

In [ ]:
!mv metagenomics/assembly/....oldname  metagenomics/assembly/....newname

Check if the rename worked:

In [ ]:
!ls metagenomics/assembly/

Did your assembly downloaded correctly?<br>
Does the content of the fasta make sense?<br>

<h2>Length distributions of the scaffolds</h2>
Now that we have the assembly, we will do some quick analyses to get a idea of the quality. First we will plot the length distribution of the scaffolds in the assembly. Luckily for us, the length of each sequence in the fasta is already embeded in each fasta header. We can easily extract these numbers and plot them in python.

To plot the length distribution run the python code below.

In [14]:
import matplotlib.pyplot as plt
import re
%matplotlib inline  
plt.style.use('ggplot')


f = open("metagenomics/assembly/scaffolds.fasta", "r")
lines = f.readlines()
f.close()


data = []
regexp = re.compile(">")

for line in lines:
    if re.search(regexp, line):
        line = line.strip().split('_')
        data.append(float(line[3]))
        
    
fig = plt.figure(figsize=(10,10))
plt.hist(data, bins=100, log=True);
plt.title("length distribution scaffolds");
plt.xlabel("length");
plt.ylabel("count");

ModuleNotFoundError: No module named 'matplotlib'

Did you expect this distribution?<br>
Why would there be so many short scaffolds?<br>

<h2> Mapping reads to scaffolds with bwa (Burrows-Wheeler Aligner) and sorting with Samtools</h2>
We now have the reads and the assembly so we can start by mapping (aligning) the original reads back to the scaffolds created with the assembly. Doing this will allow us to see the read depth on the scaffolds.

The reads are paired-end, which means that for each DNA fragment, we have sequence data from both ends. The sequences are therefore stored in two separate files (one for the data from each end). We’ll use bwa’s default settings to align the reads back and then Samtools allows us to work with the bwa results.

<b>Assignment:</b><br>
Use bwa to backmap the reads against the scaffolds and save the mapping as a .bam file.
(because we want to map all the reads back we helped you a bit with the commands).

(1) To run bwa we first need to make an index file of the scaffolds. <br>
(2) Then we can run bwa and immediatly pipe the output of bwa through samtools view which will output a bam file <br>
(3) Finally the bam file has to be sorted using Samtools sort.


some background: By default mapping algorithms like bwa spit out a .sam file. The sam format is widely used and accepted by many different downstream programmes. Basically it's just a big table with a standardised format. Although a sam file is human readable, it is also rather bulky. The filesize of a single SAM file will quickly exceed the disk space you have on this virtual machine. Therefore, we convert it to a BAM file, which is a Binary sAM file. There is no loss of information, it is just stored much more efficiently. Naturally, a binary file is not human readable. Always store your files as bam files, if you need to have a look in the bam file you can view them using samtools view or any of the many downstream programmes available online.

bwa help page:

In [ ]:
!bwa

samtools help page:

In [ ]:
!samtools

<b>run bwa index on default to make index files of your scaffolds (so no need for options)</b>

bwa index help page:

In [ ]:
!bwa index

run bwa index, finish the code: 

In [ ]:
!bwa index ....

<b>check the output of bwa</b><br>
there should now be multiple files in the assembly folder <br>
think about what command you could use to look at the files in a directory

In [ ]:
!

<b>run bwa mem for backmapping</b>

For this part we will use a bash for loop to run bwa mem and samtools view on all the reads in metagenomics/reads and  create BAM output files in a newly made directory

Then we use a bash for loop to run samtools sort on all the files created by samtools view

Step by step instruction:<br>
(1) make a new directory for the samtools view results, give this a usefull name<br>
(2) finish the for loop and run both bwa mem and samtools view<br>
(3) make a new directory for the samtools sort results, give this a usefull name<br>
(4) finish the for loop and run samtools sort<br>

bwa mem help page:

In [ ]:
!bwa mem

samtools view help page:

In [ ]:
!samtools view

In [ ]:
%%bash
for i in metagenomics/reads/*conc* ;
    do bwa mem .... "$i" | samtools view .... - > ....yourdirectoryname/"${i##*/}".bam ; 
done

samtools sort help page:

In [ ]:
!samtools sort

In [ ]:
%%bash
mkdir metagenomics/sorted

for i in ....yourdirectoryname/*.bam ;
    do samtools sort -l 5  "$i" ....yourdirectoryname/"${i##*/}".sorted ;
done

remove the samtools view directory:

In [ ]:
!rm -rf ....yourdirectoryname

samtools help page:

In [ ]:
!samtools view

run samtools view to view your BAM files:

In [ ]:
!samtools view ....yourbamsortedbam | head

<h2> Binning with MetaBat </h2>

Now that we have the reads, the scaffolds and the mapping of the reads on the scaffolds we can continue with the binning. In metagenomics, binning is the process of grouping reads or contigs and assigning them to operational taxonomic units. Binning methods can be based on either compositional features or alignment (similarity), or both. Metabat uses both the contig depth and tetra-nucleotide frequencies to bin the contigs. Every bin will represent one operational taxonomic unit that can be found in the metagenome.

<b>Assignment:</b><br>
Run the script provided in /metagenomics/scripts/jgi_summarize_bam_contig_depths to calculate the average depth per contig. <br>
Then run metabat to bin the contigs, dont forget to include the previously included contig depths in the metabat command.

In [ ]:
!./metagenomics/scripts/jgi_summarize_bam_contig_depths -h

calculate the contig depth per scaffold<br>
hint: a glob looks like this directoryname/* and includes all files included in the directory

In [ ]:
!./metagenomics/scripts/jgi_summarize_bam_contig_depths --outputDepth 

metabat help page:

In [ ]:
!metabat -h

make a new directory:

In [ ]:
!mkdir

run metabat:

In [ ]:
!metabat -i -o -t

<h2> Checking completeness and contamination using CheckM </h2>

Now that we have the bins made with metabat we can check them for contamination and completeness (quality), for this we will use CheckM. CheckM provides a set of tools for assessing the quality of genomes recovered from isolates, single cells, or metagenomes. It provides robust estimates of genome completeness and contamination by using collocated sets of genes that are ubiquitous and single-copy within a phylogenetic lineage (also called marker/signature genes). http://ecogenomics.github.io/CheckM/

As you will be able to see in the checkm help pages, checkm has a workflow (lineage_wf) that will run all nessecary steps to asses bin quality. 

Lineage_wf (lineage-specific workflow) steps: <br>
- The tree command places genome bins into a reference genome tree. <br>
- The lineage_set command creates a marker file indicating lineage-specific marker sets suitable for evaluating each genome. <br>
- This marker file is passed to the analyze command in order to identify marker genes and estimate the completeness and contamination of each genome bin.  <br>
- Finally, the qa command can be used to produce different tables summarizing the quality of each genome bin. <br>


<b>Assignment:</b><br>
Sadly for this exercise the virtual machines we are using are not powerfull enough, therefore we provide the results of the first steps of the CheckM workflow up until the qa command (lineage_wf). Scan the help pages of CheckM to find out the correct command to finish the CheckM analysis. 

(OPTIONAL: you can try to find out what the limiting factor is of lineage_wf using !/usr/bin/time --verbose) <br>

In [ ]:
!checkm -h

In [ ]:
!checkm lineage_wf -h

checkm qa help page:

In [ ]:
!checkm qa -h

run checkm qa:

In [ ]:
!checkm qa

(think and discuss these questions) <br>

What did you do? <br>
Where is your output? <br>
What does your output look like? <br>
What can you say about the bins with this output? <br>
What can you say about lineages of the bins?<br>

<h2>Genome annotation with Prokka</h2>

Now that we some extra information about our bins, we can continue to analyze the high quality bins. The final CheckM results will give you a good overview of the bins with low contamination and high completeness and also shows the lowest taxonomic rank of the bin. Pick a bin that you think is interesting to further study.

With this bin we are going to do some genome annotation. Whole genome annotation is the process of identifying features of interest in a set of genomic DNA sequences, and labelling them with useful information. Prokka is a software tool to annotate bacterial, archaeal and viral genomes quickly and produce standards-compliant output files.

<b>Assignment:</b><br>
Have a look at the prokka help pages below and come up with the right command to run prokka. (HINT: look at the Usage tag to run prokka on default mode, think of threads, use --centre X --compliant to stop prokkas complaints about ugly contig names, direct the output towars metagenomics/prokka)

In [ ]:
!prokka

make a directory for the prokka output:

In [ ]:
!mkdir

prokka help page:

In [ ]:
!prokka -h

run prokka:

In [ ]:
!prokka --cpus --outdir 

(think and discuss these questions) <br>

What did you do? <br>
Where is your output? <br>
What does your output look like? <br>
Do you know what all the output files are or mean? <br>

(HINT: you can look at the prokka files in the same way we looked at the reads and scaffold earlier)

<b>investigating prokka output</b><br>
to investigate the prokka output you can use two webservers that both are able to place the annotations from prokka in KEGG pathways.<br>

(1) prokka gives uniprot IDs in the gff files first we will collect these IDs,
(2) since we have Uniprot IDs from prokka we are going to convert these to KO IDs that can be used by KEGG using this website:<br>
http://www.uniprot.org/uploadlists/

(3) then you can put your IDs in both websites and investigate the pathways
http://www.genome.jp/kegg/tool/map_module.html<br>
http://pathways.embl.de/iPath2.cgi#<br>

view prokka output:

In [ ]:
!less ....yourprokkaoutput.gff | head

take the uniprot IDs out of the gff file

In [ ]:
!grep -o 'UniProt.*' ....yourprokkaoutput | cut -d';' -f1 | cut -d':' -f2 > listofuniprotIDs